In [ ]:
# AI-Driven Delivery Risk Triage & Operational Prioritisation

#This project demonstrates how AI can be used to support operational control by automatically identifying delivery items that genuinely require human intervention.

In [1]:
#Importing key packages & data

import pandas as pd
from datetime import datetime
import os

df = pd.read_csv(r"", parse_dates=["last_update_date"])

today = pd.Timestamp(datetime.today().date())
df["days_since_update"] = (today - df["last_update_date"]).dt.days

df

C:\Users\CHAN9473\Anaconda3\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '15/12/2025' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
C:\Users\CHAN9473\Anaconda3\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '20/01/2026' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(


,item_id,programme,work_type,priority,status,days_in_status,sla_days,risk_flag,dependencies_blocked,owner_role,last_update_date,delivery_confidence,days_since_update
0,DEL-001,Secure Systems,Milestone,High,In Progress,42,30,Y,2,Delivery Manager,2026-05-01,Low,-76
1,DEL-002,Utilities Modernisation,Incident,Medium,Open,8,10,N,0,Ops Lead,2026-01-02,High,43
2,DEL-003,Defence Platform,Milestone,Critical,At Risk,65,45,Y,3,Programme Lead,2025-12-15,Low,61
3,DEL-004,Digital Services,Change Request,Low,In Progress,12,20,N,1,Product Owner,2026-02-02,Medium,12
4,DEL-005,Secure Systems,Incident,High,Open,18,7,Y,0,Ops Lead,2026-01-20,Low,25
5,DEL-006,Utilities Modernisation,Milestone,High,At Risk,33,30,Y,1,Delivery Manager,2026-10-01,Medium,-229
6,DEL-007,Defence Platform,Incident,Critical,Open,5,3,Y,0,Ops Lead,2026-03-02,Low,-16


In [2]:
#Defining score logic

def calculate_urgency(row):
    score = 0

    if row["priority"] == "Critical":
        score += 30
    elif row["priority"] == "High":
        score += 20

    if row["days_in_status"] > row["sla_days"]:
        score += 25
    elif row["days_in_status"] > row["sla_days"] * 0.8:
        score += 15

    if row["risk_flag"] == "Y":
        score += 15

    if row["dependencies_blocked"] > 0:
        score += 10

    if row["delivery_confidence"] == "Low":
        score += 10

    if row["days_since_update"] > 14:
        score += 10

    return min(score, 100)

df["urgency_score"] = df.apply(calculate_urgency, axis=1)

df["intervention_level"] = pd.cut(
    df["urgency_score"],
    bins=[-1, 40, 70, 100],
    labels=["Monitor", "Priority Attention", "Immediate Intervention"]
)

df[["item_id", "urgency_score", "intervention_level"]]

,item_id,urgency_score,intervention_level
0,DEL-001,80,Immediate Intervention
1,DEL-002,10,Monitor
2,DEL-003,100,Immediate Intervention
3,DEL-004,10,Monitor
4,DEL-005,80,Immediate Intervention
5,DEL-006,70,Priority Attention
6,DEL-007,80,Immediate Intervention


In [3]:
intervention_queue = df[df["intervention_level"] != "Monitor"]
[
    [
        "item_id",
        "programme",
        "work_type",
        "priority",
        "status",
        "urgency_score",
        "intervention_level",
        "owner_role"
    ]
].sort_values(by="urgency_score", ascending=False)

intervention_queue

,item_id,programme,work_type,priority,status,urgency_score,intervention_level,owner_role
2,DEL-003,Defence Platform,Milestone,Critical,At Risk,100,Immediate Intervention,Programme Lead
0,DEL-001,Secure Systems,Milestone,High,In Progress,80,Immediate Intervention,Delivery Manager
4,DEL-005,Secure Systems,Incident,High,Open,80,Immediate Intervention,Ops Lead
6,DEL-007,Defence Platform,Incident,Critical,Open,80,Immediate Intervention,Ops Lead
5,DEL-006,Utilities Modernisation,Milestone,High,At Risk,70,Priority Attention,Delivery Manager


In [4]:
prompt = f"""
You are an operational delivery advisor supporting leadership intervention.

The following delivery items require attention:
{intervention_queue.to_string(index=False)}

For each item, explain:
1. Why intervention is required
2. What type of action is appropriate (exec escalation, operational fix, dependency resolution)
3. The risk of inaction

Keep the tone concise and operational.
"""

print(prompt)


You are an operational delivery advisor supporting leadership intervention.

The following delivery items require attention:
item_id               programme work_type priority      status  urgency_score     intervention_level       owner_role
DEL-003        Defence Platform Milestone Critical     At Risk            100 Immediate Intervention   Programme Lead
DEL-001          Secure Systems Milestone     High In Progress             80 Immediate Intervention Delivery Manager
DEL-005          Secure Systems  Incident     High        Open             80 Immediate Intervention         Ops Lead
DEL-007        Defence Platform  Incident Critical        Open             80 Immediate Intervention         Ops Lead
DEL-006 Utilities Modernisation Milestone     High     At Risk             70     Priority Attention Delivery Manager

For each item, explain:
1. Why intervention is required
2. What type of action is appropriate (exec escalation, operational fix, dependency resolution)
3. The risk o

In [5]:
os.makedirs("outputs", exist_ok=True)

with open("", "w") as file:
    file.write(prompt)

print("Operational intervention queue saved to outputs/intervention_queue.txt")

Operational intervention queue saved to outputs/intervention_queue.txt
